In [1]:
import numpy as np
import math
from pypoman import compute_polytope_vertices

np.random.seed(3)

# Querying to Estimate - Linear Additive Domain (LAD) Application

                                   Adel Magra, Tim Baarslag, Michael Kaisers

We are interested in finding an optimal sequence of queries to ask a user to estimate their utility. Optimal is meant in the sense that the sequence should provide the most accurate of estimation for the least query cost. 

## The Domain

The negotiation domain $\Omega$ is a "multiple split the pie" domain. There are $n$ pies that need to be shared between 2 negotiators $N_1$ and $N_2$. We let then $\Omega = [0,1]^n$, where each outcome $\omega\in\Omega$ corresponds to the amount of each pie given to $N_1$. For example $\omega = (1,1\cdots,1,0)$ means $N_1$ gets all the first $n-1$ pies and the last pie goes to $N_2$. $\omega = (0.5,0.5,\cdots,0.5)$ means each negotiator gets half of each pie. Here is an illustration for $n=3$ of the outcome $(0.4,0.3,0.5)$ 

<img src="Images/pies.png" alt="Drawing" style="width: 500px;"/>


  
   
## The Model

Assume we are the agent that is representing $N_1$. It is safe to assume that $N_1$ wants more of any given pie. However, the importance attached to getting a particular pie may differ. For this reason, we model the utility of $N_1$ with the following linear addtive utiltiy functions $\mathcal{U}$:
    
$$\mathcal{U}=\left\{\omega\mapsto\sum_{i=1}^n \theta_i\cdot \omega_i | \theta_i\geq 0 \forall i, \sum_{i=1}^n \theta_i = 1\right\}$$

We assume that the true utility $u^*$ of $N_1$ is in $\in\mathcal{U}$. It follows that we are looking for the corresponding vector of parameters $\theta^*\in\Theta:=\{\theta\in\mathbb[0.1]^n | \sum_{i=1}^n\theta_i=1\}$.



In [2]:
n = 3 # Number of pies
true_theta = np.random.dirichlet([1 for i in range(n)]) # Generate the true parameter uniformly at random

'''Computes the utility of an outcome.'''
def utility(outcome, theta):
    return sum([outcome[i]*theta[i] for i in range(n)])

print(true_theta)

[0.33688219 0.51840834 0.14470947]


## Querying

It is possible to query the user for information on $u^*$ against a cost. The queries considered are pairwise comparisons of outcomes, namely: 

$$\mathcal{Q}=\{(\omega\geq \omega')? | \omega,\omega\in\Omega\}$$

where $(\omega\geq \omega')?$ is to be understood as: "Is $u^*(\omega)\geq u^*(\omega')$ ?" or "Do you prefer outcome $\omega$ over outcome $\omega'$?" The possible answers are simply 1 (Yes) if indeed $\omega$ is prefered, or 0 (No) otherwise. We also assume constant cost of queries. Namely, for each $q\in\mathcal{Q}, c(q) = c$

In [3]:
query_cost = 0.01 # Constant cost initialization
cum_cost = 0 # Cummulative cost

def query(outcome_1, outcome_2):
    # Update cumulative cost
    global cum_cost
    global query_cost
    cum_cost += query_cost
    
    # Compute the true utilities of the outcomes and compare them
    if(utility(outcome_1, true_theta) >= utility(outcome_2,true_theta)):
        return 1
    return 0

#EXAMPLE#
outcome_1 = [0,0.5,1]
outcome_2 = [1,0.5,0]
print("theta =", true_theta)
print("u((0,0.5,1)) = ", utility(outcome_1,true_theta))
print("u((1,0.5,0)) = ", utility(outcome_2,true_theta))

if(query(outcome_1,outcome_2)):
    print("(0,0.5,1) is prefered over (1,0.5,0)")
else:
    print("(1,0.5,0) is prefered over (0,0.5,1)")

#Reset cum_cost after example
cum_cost = 0

theta = [0.33688219 0.51840834 0.14470947]
u((0,0.5,1)) =  0.4039136405805671
u((1,0.5,0)) =  0.596086359419433
(1,0.5,0) is prefered over (0,0.5,1)


## Estimation

We are allowed to query the user in order to help us achieve obtain an accurate estimation. We want to provide $\widehat{\theta}$ that minimizes the worse case loss function. The reason we deal with worse case is because after having asked a bunch of queries $Q$, we only know that $\theta^*\in S_{a^*(Q)}$ where $S_{a^*(Q)}$ corresponds to the $\theta\in\Theta$ whose answers to queries in $Q$ are in concordance with the answers $a^*(Q)$ received.

### Loss Function

The loss function is the squared Euclidean distance:

$$\ell(\theta,\theta') = \sum_{i=1}^n(\theta_i-\theta'_i)^2$$

The worse case loss over a set $S\subseteq \Theta$ is then:

$$L(\theta,S) = \max_{\theta'\in S} \ell(\theta,\theta')$$


In [4]:
def loss(theta_1, theta_2):
    return sum([(theta_1[i]-theta_2[i])*(theta_1[i]-theta_2[i]) for i in range(n)])

#EXAMPLE#
# Select two random thetas
theta_1 = np.random.dirichlet([1 for i in range(n)])
theta_2 = np.random.dirichlet([1 for i in range(n)])

print("theta_1 = ", theta_1)
print("theta_2 = ", theta_2)
print("l(theta_1,theta_2) = ", loss(theta_1,theta_2))

theta_1 =  [0.13709495 0.42840826 0.43449679]
theta_2 =  [0.32007545 0.55390092 0.12602362]
l(theta_1,theta_2) =  0.1443859697647191


### Centroid Finder: the min-max Estimation

Observe that $S_{a^*(Q)}$ is a closed convex polytope of dimension $n-1$. For example, the first set of possible thetas $\Theta = S_{a^*(\varnothing)}$ is the $n-1$ standard simplex $\Delta^{n-1}$. Here is what $\Theta$ looks like when $n=3$:

<img title="Theta in R3" src="Images/std_simplex_3d.png" alt="Drawing" style="width: 500px;"/>


The description we have of the convex polytope $S_{a^*(Q)}$ is the $H-$description ($H$ for half-space). This goes because each answered query $q\in Q$ can be seen to correspond to an inequality $u^*(\omega_q)\geq u^*(\omega_q')$ which can be carefully rewritten as $\sum_{i=1}^n x_{q,i} \theta^*_i\leq 0$ where $x_{i,q}\in\mathbb{R}$. Therefore, we have the following H-description of $S_{a^*(Q)}$:

\begin{equation}
\begin{split}
    &\theta \in S_{a^*(Q)} \Longleftrightarrow\\
    &\forall i, \theta_i\geq 0\\
    &\sum_{i=1}^n\theta_i = 1\\
    &\forall q\in Q, \sum_{i=1}^n x_{q,i} \theta_i\leq 0
\end{split}
\end{equation}

The worse case euclidean distance minimizer for a convex polytope is $\textbf{not obvious}$. But for regular polytopes, it is the centroid point. Given the vertices $v_1,\cdots,v_k$ of a polytope $P$, its centroid is defined as the average of those points:

$$C = \dfrac{1}{k} \sum_{i=1}^k v_i$$

How do we compute The centroid of $S_{a^*(Q)}$ since we only have the $H$-description of $S_{a^*(Q)}$? Well, we need to find the $V$-description ($V$ for vertex) of $S_{a^*(Q)}$. This is possible thanks to the $\textbf{compute_polytope_vertices}$ function of the package $\textbf{pypoman}$ by Stéphane Caron.

We are now ready to compute the worse-case loss $L(\widehat{\theta},S_{a^*(Q)})$ for any given set $S_{a^*(Q)}$.

In [12]:
'''
INPUT: a list of answered queries
An answered query is a 3-tuple of this shape (o_1, o_2, query(o_1,o_2))

OUTPUT: The vertex representation of the polytope defined by the answered queries. 
'''
def vertex_rep(answered_queries): 
    m = len(answered_queries)
    
    # Initialize the input 
    A = [[0]*n for i in range((n+2)+m)]
    b = [0]*((n+2)+m)
    
    # Start with the basic n+2 conditions: -theta_i <= 0 and sum(theta_i) <= 1 and sum(-theta_i) <= -1
    for i in range(n):
        A[i][i] = -1
        A[n][i] = 1
        A[n+1][i] = -1
    b[n] = 1
    b[n+1] = -1
    
    # Continue by converting the answered queries into a valid input for compute_polytope_vertices
    for i in range(m):
        o_1 = answered_queries[i][0]
        o_2 = answered_queries[i][1]
        answer = answered_queries[i][2]
        
        if answer == 1: # Taking care of the inequality sign of u^*(o_1) >= u^*(o_2)
            A[i+n+2] = [o_2[j]-o_1[j] for j in range(n)] 
        else:
            A[i+n+2] = [o_1[j]-o_2[j] for j in range(n)]
    
    # Finish by computing the vertices
    return compute_polytope_vertices(np.array(A),np.array(b))
  
'''
INPUT: a list of answered queries.

OUTPUT: The centroid and the worse case loss.
'''
def worse_loss(answered_queries):
    v = vertex_rep(answered_queries)
    c = sum([p/len(v) for p in v])
    
    #Compute max loss of centroid to other vertices
    w = max([loss(c,p) for p in v])
    
    return (c,w)



worse_loss([([1,0.5,1],[1,1,0.5],1),([0,0.5,1],[1,0.7,0.5],0), ([1,0.5,0],[1,0,0.5],1)])

(array([0.56521739, 0.2173913 , 0.2173913 ]), 0.28355387523629494)

## Querying Algorithms

Given a set of answered queries $a^*(Q)$, we can compute its corresponding polytope $S_{a^*(Q)}$. The question we pose is: What is the next query we should ask? Well, we would like to ask the query that minimzes the worse case loss of our estimation on the updated polytope when answered. Namely, we want to find $q^*$ such that:

$$q^* = \text{argmin}_{q\in \mathcal{Q}}\mathbb{E}_{a^*(q)}[L(\widehat{\theta},S_{a^*(Q)\cup a^*(q)}]$$

Assuming that queries have a 50-50 chance of being answered Yes or No, we want to find $q^*$ s.t.:

$$q^* = \text{argmin}_{q\in \mathcal{Q}} \dfrac{1}{2}L(\widehat{\theta},S_{a^*(Q)\cup\{(q,1)\})}) + \dfrac{1}{2}L(\widehat{\theta},S_{a^*(Q)\cup\{(q,0)\})})$$

In [5]:
'''
INPUT: list of answered queries Q, query q.

OUTPUT: expected worse case loss of centroid estimation when adding q to Q.
'''
def expected_worse_loss(answered_queries, q):

    return 0
    

### Optimal Querying ($n\in \{2,3\}$)

For $n=2$, we begin with $\Theta$ being the line segment $\theta_1+\theta_2 = 1, \theta_1\geq 0, \theta_2\geq 0$ in $\mathbb{R}^2$. Each query will cut this segment in two segments. Observe that the centroid of any segment is its midpoint. It follows that the worse-case loss of any segment is equal to half its length. (Note that it is also true that the centroid is the estimator that minimizes the worse case loss).

It is easy to see that given a segment, the optimal next query to ask is the one which cuts it in half. We describe this below

In [6]:
'''
INPUT: A list of answered queries for a domain with 2 pies only, i.e. n=2

OUTPUT: A tuple of the best next query to ask and its expected worse case loss
'''

def opt_query_2(answered_queries):
    return 0

### Approximation of Optimal Querying
 
For a general number of pies $n\geq 3$, it becomes harder to find the optimal query analytically. What we rather do is discretize the query space to $\mathcal{Q}_m$:

$$\mathcal{Q}_m = \left\{(\omega_1,\cdots,\omega_n)\geq (\omega'_1,\cdots \omega'_n)? | \forall i, \omega_i,\omega'_i\in \left\{0,\dfrac{1}{m},\cdots,1\right\}\right\}$$

We then do iterate over $\mathcal{Q}_m$ to find the optimal next query. 

In [7]:
'''
INPUT: A list of answered queries over a domain with any number of pies

OUTPUT: A tuple of: (Approximation of the next best query to ask, its expected worse-case loss)
'''

def approximate_opt_query(answered_queries):
    return 0

### MRS-Bound

## Experiments